In [1]:
#IMPORTS

# Libraries to help with reading and manipulating data
import numpy as np
import pandas as pd

# Libraries to help with data visualization
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()

# split the data into train and test
from sklearn.model_selection import train_test_split

# to build linear regression_model
from sklearn.linear_model import LinearRegression

# to check model performance
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# to build linear regression_model using statsmodels
import statsmodels.api as sm

# to compute VIF
from statsmodels.stats.outliers_influence import variance_inflation_factor

### Acquire Data

In [2]:
# acquire data
data = pd.read_csv('used_device_data.csv') 

### What does our data look like inititally?

In [3]:
#first glimpses of data
data.head()

,brand_name,os,screen_size,4g,5g,main_camera_mp,selfie_camera_mp,int_memory,ram,battery,weight,release_year,days_used,normalized_used_price,normalized_new_price
0,Honor,Android,14.50,yes,no,13.0,5.0,64.0,3.0,3020.0,146.0,2020,127,4.307572,4.715100
1,Honor,Android,17.30,yes,yes,13.0,16.0,128.0,8.0,4300.0,213.0,2020,325,5.162097,5.519018
2,Honor,Android,16.69,yes,yes,13.0,8.0,128.0,8.0,4200.0,213.0,2020,162,5.111084,5.884631
3,Honor,Android,25.50,yes,yes,13.0,8.0,64.0,6.0,7250.0,480.0,2020,345,5.135387,5.630961
4,Honor,Android,15.32,yes,no,13.0,8.0,64.0,3.0,5000.0,185.0,2020,293,4.389995,4.947837


In [4]:
data.tail()

,brand_name,os,screen_size,4g,5g,main_camera_mp,selfie_camera_mp,int_memory,ram,battery,weight,release_year,days_used,normalized_used_price,normalized_new_price
3449,Asus,Android,15.34,yes,no,NaN,8.0,64.0,6.0,5000.0,190.0,2019,232,4.492337,6.483872
3450,Asus,Android,15.24,yes,no,13.0,8.0,128.0,8.0,4000.0,200.0,2018,541,5.037732,6.251538
3451,Alcatel,Android,15.80,yes,no,13.0,5.0,32.0,3.0,4000.0,165.0,2020,201,4.357350,4.528829
3452,Alcatel,Android,15.80,yes,no,13.0,5.0,32.0,2.0,4000.0,160.0,2020,149,4.349762,4.624188
3453,Alcatel,Android,12.83,yes,no,13.0,5.0,16.0,2.0,4000.0,168.0,2020,176,4.132122,4.279994


In [5]:
#How many rows and columns?
data.shape

(3454, 15)

In [6]:
#Here we learn the column names, if there are nulls, and the data type of each column
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3454 entries, 0 to 3453
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   brand_name             3454 non-null   object 
 1   os                     3454 non-null   object 
 2   screen_size            3454 non-null   float64
 3   4g                     3454 non-null   object 
 4   5g                     3454 non-null   object 
 5   main_camera_mp         3275 non-null   float64
 6   selfie_camera_mp       3452 non-null   float64
 7   int_memory             3450 non-null   float64
 8   ram                    3450 non-null   float64
 9   battery                3448 non-null   float64
 10  weight                 3447 non-null   float64
 11  release_year           3454 non-null   int64  
 12  days_used              3454 non-null   int64  
 13  normalized_used_price  3454 non-null   float64
 14  normalized_new_price   3454 non-null   float64
dtypes: f

In [7]:
# This gives us the statistical summary of thw numerical columns
data.describe()

,screen_size,main_camera_mp,selfie_camera_mp,int_memory,ram,battery,weight,release_year,days_used,normalized_used_price,normalized_new_price
count,3454.000000,3275.000000,3452.000000,3450.000000,3450.000000,3448.000000,3447.000000,3454.000000,3454.000000,3454.000000,3454.000000
mean,13.713115,9.460208,6.554229,54.573099,4.036122,3133.402697,182.751871,2015.965258,674.869716,4.364712,5.233107
std,3.805280,4.815461,6.970372,84.972371,1.365105,1299.682844,88.413228,2.298455,248.580166,0.588914,0.683637
min,5.080000,0.080000,0.000000,0.010000,0.020000,500.000000,69.000000,2013.000000,91.000000,1.536867,2.901422
25%,12.700000,5.000000,2.000000,16.000000,4.000000,2100.000000,142.000000,2014.000000,533.500000,4.033931,4.790342
50%,12.830000,8.000000,5.000000,32.000000,4.000000,3000.000000,160.000000,2015.500000,690.500000,4.405133,5.245892
75%,15.340000,13.000000,8.000000,64.000000,4.000000,4000.000000,185.000000,2018.000000,868.750000,4.755700,5.673718
max,30.710000,48.000000,32.000000,1024.000000,12.000000,9720.000000,855.000000,2020.000000,1094.000000,6.619433,7.847841


In [8]:
# make a copy of data,creating a copy of the data so that original data remains unchanged
df = data.copy()

### Prep and Clean Data

#### To get the data ready for EDA, I will check the following (if needed):
- Deal with Missing Values
- Remove Duplicate/Low Variation Data (1 or little unique values)
- Incorrect/Irrelevant Data (fix structural errors)
- Categorical Data and Type Conversion
- Handle Outliers
- Feature Scaling
- Feature Engineering/Selection

#### Deal with missing data
#### Do I want to filter out or fill in missing data?

In [9]:
# There are not too many missing values.  
df.isnull().sum()

brand_name                 0
os                         0
screen_size                0
4g                         0
5g                         0
main_camera_mp           179
selfie_camera_mp           2
int_memory                 4
ram                        4
battery                    6
weight                     7
release_year               0
days_used                  0
normalized_used_price      0
normalized_new_price       0
dtype: int64

In [10]:
# main_camera_mp is missing 179
3454-179

3275

In [11]:
# That is only about 5%
3275/3454

0.9481760277938622

In [12]:
# Since it is a small amount of missing,  I will filter out or drop, rows with missing data 
df = df.dropna() # drop all rows with NaN

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3253 entries, 0 to 3453
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   brand_name             3253 non-null   object 
 1   os                     3253 non-null   object 
 2   screen_size            3253 non-null   float64
 3   4g                     3253 non-null   object 
 4   5g                     3253 non-null   object 
 5   main_camera_mp         3253 non-null   float64
 6   selfie_camera_mp       3253 non-null   float64
 7   int_memory             3253 non-null   float64
 8   ram                    3253 non-null   float64
 9   battery                3253 non-null   float64
 10  weight                 3253 non-null   float64
 11  release_year           3253 non-null   int64  
 12  days_used              3253 non-null   int64  
 13  normalized_used_price  3253 non-null   float64
 14  normalized_new_price   3253 non-null   float64
dtypes: f

### Remove duplicate rows

In [13]:
#Drop duplicates
df.drop_duplicates()

,brand_name,os,screen_size,4g,5g,main_camera_mp,selfie_camera_mp,int_memory,ram,battery,weight,release_year,days_used,normalized_used_price,normalized_new_price
0,Honor,Android,14.50,yes,no,13.0,5.0,64.0,3.0,3020.0,146.0,2020,127,4.307572,4.715100
1,Honor,Android,17.30,yes,yes,13.0,16.0,128.0,8.0,4300.0,213.0,2020,325,5.162097,5.519018
2,Honor,Android,16.69,yes,yes,13.0,8.0,128.0,8.0,4200.0,213.0,2020,162,5.111084,5.884631
3,Honor,Android,25.50,yes,yes,13.0,8.0,64.0,6.0,7250.0,480.0,2020,345,5.135387,5.630961
4,Honor,Android,15.32,yes,no,13.0,8.0,64.0,3.0,5000.0,185.0,2020,293,4.389995,4.947837
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3447,Apple,iOS,15.27,yes,no,8.0,7.0,64.0,4.0,3110.0,194.0,2019,208,5.100902,6.287933
3450,Asus,Android,15.24,yes,no,13.0,8.0,128.0,8.0,4000.0,200.0,2018,541,5.037732,6.251538
3451,Alcatel,Android,15.80,yes,no,13.0,5.0,32.0,3.0,4000.0,165.0,2020,201,4.357350,4.528829
3452,Alcatel,Android,15.80,yes,no,13.0,5.0,32.0,2.0,4000.0,160.0,2020,149,4.349762,4.624188


### Good, no dups!

In [ ]:

Incorrect/Irrelevant Data (fix structural errors)


In [ ]:
Categorical Data and Type Conversion


In [ ]:
Handle Outliers


In [ ]:
Feature Scaling


In [ ]:
Feature Engineering/Selection

In [42]:
data.duplicated().sum()

0

In [41]:
(df.drop_duplicates()).shape

(3253, 15)

In [17]:
df.head()

,brand_name,os,screen_size,4g,5g,main_camera_mp,selfie_camera_mp,int_memory,ram,battery,weight,release_year,days_used,normalized_used_price,normalized_new_price
0,Honor,Android,14.50,yes,no,13.0,5.0,64.0,3.0,3020.0,146.0,2020,127,4.31,4.72
1,Honor,Android,17.30,yes,yes,13.0,16.0,128.0,8.0,4300.0,213.0,2020,325,5.16,5.52
2,Honor,Android,16.69,yes,yes,13.0,8.0,128.0,8.0,4200.0,213.0,2020,162,5.11,5.88
3,Honor,Android,25.50,yes,yes,13.0,8.0,64.0,6.0,7250.0,480.0,2020,345,5.14,5.63
4,Honor,Android,15.32,yes,no,13.0,8.0,64.0,3.0,5000.0,185.0,2020,293,4.39,4.95


In [ ]:
4g	5g yes no

In [18]:
df.release_year.min()

2013

In [19]:
df.release_year.max()

2020

In [20]:
df.brand_name.unique()

array(['Honor', 'Others', 'HTC', 'Huawei', 'Lava', 'Lenovo', 'LG',
       'Micromax', 'Nokia', 'Oppo', 'Samsung', 'Vivo', 'Xiaomi', 'ZTE',
       'Apple', 'Asus', 'Acer', 'Alcatel', 'BlackBerry', 'Celkon',
       'Coolpad', 'Gionee', 'Google', 'Karbonn', 'Meizu', 'Microsoft',
       'Motorola', 'OnePlus', 'Panasonic', 'Realme', 'Sony', 'Spice',
       'XOLO'], dtype=object)

In [21]:
df.os.unique()

array(['Android', 'Others', 'iOS', 'Windows'], dtype=object)

In [22]:
#gonna need to bin theses
df.screen_size.unique()

array([14.5 , 17.3 , 16.69, 25.5 , 15.32, 16.23, 13.84, 15.77, 15.47,
       15.9 , 16.74, 25.43, 20.32, 15.29, 12.88, 15.24, 15.37, 16.71,
       25.6 , 15.34, 16.59, 16.81, 20.42, 10.16, 17.78, 12.7 , 15.44,
        5.28, 15.27, 15.8 , 15.72, 12.83,  7.62,  7.75, 10.03, 16.56,
        5.18, 15.39, 10.29, 16.43, 12.93, 15.42, 16.36, 16.21, 16.13,
       17.5 , 16.28, 14.4 , 15.21, 20.12, 16.48, 16.89, 16.31, 25.53,
       12.73, 20.55, 30.71, 27.94, 25.45, 18.01, 14.35, 23.04, 10.34,
       10.24, 12.78, 12.75, 23.01, 11.81, 11.84, 10.36, 11.76, 12.9 ,
       17.4 , 14.55, 12.8 ,  7.65, 10.21,  5.08,  5.13,  5.23, 20.35,
       12.57, 16.33, 18.08, 17.65, 14.83, 15.06, 13.08, 15.14, 30.56,
       19.96, 19.94, 25.4 , 11.89, 15.62, 17.83,  7.85,  7.67, 16.08,
       17.73, 12.52, 20.4 ,  7.82, 15.11, 13.87,  5.16, 10.08, 15.7 ,
       14.88, 15.82, 15.95,  7.7 , 14.53, 17.86, 12.85, 13.34, 11.48,
        7.98, 30.53,  7.8 , 12.12,  8.31,  6.68, 10.31, 11.56, 14.86,
       14.94, 13.79,

In [24]:
df.columns

Index(['brand_name', 'os', 'screen_size', '4g', '5g', 'main_camera_mp',
       'selfie_camera_mp', 'int_memory', 'ram', 'battery', 'weight',
       'release_year', 'days_used', 'normalized_used_price',
       'normalized_new_price'],
      dtype='object')

In [25]:
df.main_camera_mp.min()

0.08

In [26]:
df.main_camera_mp.max()

48.0

In [27]:
df.main_camera_mp.unique()

array([13.  ,  8.  ,  5.  , 10.5 ,  3.15,  2.  , 16.  ,  0.3 , 12.  ,
       14.5 , 48.  ,  3.  , 21.  ,  1.3 , 13.1 , 24.  ,  0.08, 20.7 ,
       23.  ,  1.  , 18.  , 12.2 , 12.3 , 20.  , 20.2 ,  4.  , 12.5 ,
       10.  ,  6.5 ,  6.7 , 41.  , 20.1 , 12.6 , 16.3 , 22.6 , 19.  ,
       21.5 , 21.2 ,  8.1 ,  1.2 , 22.5 ])

In [28]:
df.selfie_camera_mp.min()

0.0

In [29]:
df.selfie_camera_mp.max()

32.0

In [30]:
df.selfie_camera_mp.unique()

array([ 5.  , 16.  ,  8.  , 32.  ,  2.  ,  0.3 , 13.  , 14.5 , 24.  ,
       10.  ,  1.3 ,  0.  , 25.  , 20.  , 12.  ,  7.  ,  1.2 ,  3.  ,
        2.1 ,  1.  ,  4.  ,  1.1 , 10.5 ,  1.6 ,  1.8 ,  0.9 ,  1.25,
       18.  ,  1.9 ,  9.  ,  3.7 , 16.3 ,  5.1 ,  2.2 , 14.  ,  0.65])

In [31]:
df.int_memory.min()

0.01

In [32]:
df.int_memory.max()

1024.0

In [33]:
df.ram.min()

0.02

In [34]:
df.ram.max()

12.0

In [35]:
df.battery.min()

500.0

In [36]:
df.battery.max()

9720.0

In [37]:
df.weight.min()

69.0

In [38]:
df.weight.max()

855.0

In [39]:
df.days_used.min()

91

In [40]:
df.days_used.max()

1094

In [ ]:
'battery', 'weight',
       'release_year', 'days_used', 'normalized_used_price',
       'normalized_new_price'],

In [ ]:
# drop columns
df = df.drop('col1', axis=1)

In [ ]:
3454-179

In [ ]:
3275/3454